In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import seaborn as sns
import seaborn.objects as so

In [132]:
raw_data = pd.read_csv(r'C:\Users\mds8301\Documents\Github\credit_modeling\data\loans_2007.csv', low_memory=False)

In [137]:
def clean_loans(df):
    
    def find_cols_with_missing_data(_df, threshold):
        """Returns a list of columns with missing data over specified threshold"""
        return [col for col in _df.columns if _df[col].isnull().sum() > (_df.shape[0]*threshold)]
    
    def find_cols_with_single_val(_df):
        """ returns a list of columns with single unique values"""
        non_uniuqe_vals = _df.nunique(dropna= True) ==1 
        return [non_uniuqe_vals.index[i] for i,col in enumerate(non_uniuqe_vals) if col == True]
    
    missing_data_cols = find_cols_with_missing_data(df, threshold = 0.5)
    non_uniq_cols = find_cols_with_single_val(df)

    data_leak_cols = ['out_prncp', 'out_prncp_inv', 'total_pymnt','total_pymnt_inv', 'total_rec_prncp',
                      'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
                      'funded_amnt', 'funded_amnt_inv','issue_d', 'last_pymnt_d', 'last_pymnt_amnt'
                      ]
    
    non_useful_cols = ['title','emp_title' ]
    edited_cols = ['verification_status', 'sub_grade', 'id', 'member_id','verification_status','pymnt_plan', 'zip_code']
    
    columns_to_drop = [*missing_data_cols, *data_leak_cols, *edited_cols, *non_uniq_cols]
        
    ordinal_grade_type = pd.CategoricalDtype(categories = sorted(df.grade.dropna().unique()), ordered = True)
    
    return(
        df
        .iloc[:-2]
        .rename(columns = {'int_rate':'int_rate_percent' })
        .assign(int_rate_percent = lambda df_: df_.int_rate_percent.str.replace('%','').astype(float),
                grade = lambda df_: df_.grade.astype(ordinal_grade_type),
                emp_length = lambda df_: df_.emp_length.str.replace('\D', '', regex = True),
                not_verified = lambda df_: df_.verification_status.str.contains('[nN]ot').astype(bool),
                renter = lambda df_: df_.home_ownership.str.contains('RENT'),
                payment_plan = lambda df_: df_.pymnt_plan.str.contains('y'),
                earliest_cr_line = lambda df_: pd.to_datetime(df_.earliest_cr_line),
                revol_util = lambda df_: df_.revol_util.str.replace('%', '').astype(float)
                )
        .drop(columns = columns_to_drop)
        .dropna()
        .query("loan_status == 'Fully Paid' or loan_status == 'Charged Off'")
        .replace(
            {
            "loan_status": {"Fully Paid": 1, "Charged Off": 0},
            "term":{" 36 months": 0, " 60 months": 1}
            }
            )

        )
    


    

In [138]:
loans = clean_loans(raw_data)
loans.columns

Index(['loan_amnt', 'term', 'int_rate_percent', 'installment', 'grade',
       'emp_title', 'emp_length', 'home_ownership', 'annual_inc',
       'loan_status', 'purpose', 'title', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'open_acc', 'pub_rec',
       'revol_bal', 'revol_util', 'total_acc', 'last_credit_pull_d',
       'acc_now_delinq', 'delinq_amnt', 'pub_rec_bankruptcies', 'tax_liens',
       'not_verified', 'renter', 'payment_plan'],
      dtype='object')

In [108]:
loans.select_dtypes('object').columns

Index(['emp_title', 'emp_length', 'home_ownership', 'purpose', 'title',
       'addr_state', 'last_credit_pull_d', 'renter', 'payment_plan'],
      dtype='object')

In [139]:
loans.term

1        1
3        0
5        0
6        1
7        0
        ..
39620    0
39631    0
39692    0
39735    0
39749    0
Name: term, Length: 35610, dtype: int64